In [1]:
# !pip install -r requirements.txt

In [2]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(900_000_000_000) ## 800GB?

import geopandas as gpd
import pandas as pd

import utilities
import A1_rail_ferry_brt as rail_ferry_brt

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
analysis_date = rail_ferry_brt.analysis_date
analysis_date

datetime.date(2022, 6, 15)

[PRC 21064.3](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=21064.3.&lawCode=PRC)
* _Major transit stop means a site containing any of the following:
(a) An existing rail or bus rapid transit station.
(b) A ferry terminal served by either a bus or rail transit service.
(c) The intersection of two or more major bus routes with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods._
    * "Intersection" may not be sufficiently well-defined for this analysis

[PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit)
* _(a) “Bus rapid transit” means a public mass transit service provided by a public agency or by a public-private partnership that includes all of the following features:
(1) Full-time dedicated bus lanes or operation in a separate right-of-way dedicated for public transportation with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods.
(2) Transit signal priority.
(3) All-door boarding.
(4) Fare collection system that promotes efficiency.
(5) Defined stations._
    * Unlikely to determine if a service qualifies as BRT under this definition using GTFS alone

### Rail Service

* temporary workaround with dim tables, should use tbl.gtfs schedule but LA Metro rail not in

In [4]:
rail_stops = rail_ferry_brt.grab_rail_data(analysis_date)

In [5]:
#utilities.map_hqta(rail_stops, 'route_type')

Map(center=[35.12126000418686, -120.62926600000003], controls=(ZoomControl(options=['position', 'zoom_in_text'…

#### BRT Service likely meeting [PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit) definition:

* LA Metro Orange, Silver excluding street running (stop flags only)
* ~~Omnitrans sbX, all stops (curbside stations are well-defined, with fare prepayment)~~
    * insufficient frequency 5/16
* AC Transit Tempo, all stops (curbside stations are well-defined, with fare prepayment)

[PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit)
* _(a) “Bus rapid transit” means a public mass transit service provided by a public agency or by a public-private partnership that includes all of the following features:
(1) Full-time dedicated bus lanes or operation in a separate right-of-way dedicated for public transportation with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods.
(2) Transit signal priority.
(3) All-door boarding.
(4) Fare collection system that promotes efficiency.
(5) Defined stations._
    * Unlikely to determine if a service qualifies as BRT under this definition using GTFS alone

In [6]:
metro_brt_stops = rail_ferry_brt.grab_operator_brt(182, analysis_date)
print(f"# stops in Metro BRT: {len(metro_brt_stops)}")

# stops in Metro BRT: 98


In [7]:
# This is a manual step, check with map and figure out what goes into this list

## unable to filter out non-station stops using GTFS, manual list:
metro_street_running =[
    '141012', '13805', '5397', '13803',
    '13804', '5396', '13802', '5395', '5410', '5411', '13817',
    '12304', '5408', '3821', '2603', '3153', '3124', '378', '65300039',
    '65300038', '15820', '13460', '4994', '1813', '2378', '5049',
    '4652', '2377', '4675', '5040', '65300042', '3674', '15713',
    '13561', '5378', '13560', '70500012', '5377', '15612',
    '12416', '11917', '12415', '8704'
]

metro_brt_stops = rail_ferry_brt.additional_brt_filtering_out_stops(
    metro_brt_stops, 182, metro_street_running)

print(f"# stops in Metro BRT (after filtering): {len(metro_brt_stops)}")

# stops in Metro BRT (after filtering): 55


In [8]:
#utilities.map_hqta(metro_brt_stops)
## OK June 23

Map(center=[33.79036700366605, -118.28219100000003], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [9]:
act_brt_stops = rail_ferry_brt.grab_operator_brt(4, analysis_date)

In [10]:
#utilities.map_hqta(act_brt_stops)

Map(center=[37.721569005280124, -122.1601659999999], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [11]:
#omni_brt_stops = rail_ferry_brt.grab_operator_brt(232, analysis_date)
## too infrequent!

In [12]:
#utilities.map_hqta(omni_brt_stops)

In [13]:
muni_brt_stops = rail_ferry_brt.grab_operator_brt(282, analysis_date)
print(f"# stops in MUNI BRT: {len(muni_brt_stops)}")

# stops in MUNI BRT: 85


In [15]:
van_ness_ids = [
    '8096', '8097', '18095', '18098', '8094', '8099', '18093', '8100',
    '18092', '18101', '18102', '18091', '18103', '8090', '18104', '18089',
    '18105', '18088']

muni_brt_stops = rail_ferry_brt.additional_brt_filtering_out_stops(
    muni_brt_stops, 282, 
    van_ness_ids)

print(f"# stops in MUNI BRT (after filtering): {len(muni_brt_stops)}")

# stops in MUNI BRT (after filtering): 12


In [16]:
#utilities.map_hqta(muni_brt_stops)

Map(center=[37.78291700530728, -122.42086500000002], controls=(ZoomControl(options=['position', 'zoom_in_text'…

### Ferry

In [17]:
ferry_stops = rail_ferry_brt.grab_ferry_data(analysis_date)

In [18]:
#utilities.map_hqta(ferry_stops)

Map(center=[32.699209003255, -117.16972], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_…